## CRM Agent with ADK

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Fcrm-agent-mcp.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [56]:
%pip install google-adk==1.0.0 -q
%pip install litellm -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [57]:
import os
import sys
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams

### Set Auth

In [58]:
CLOUD_PROJECT = "GCP_PROJECT_ID" # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"]= CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"]="us-central1"

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Get Apigee MCP Tools

In [59]:
def get_customer_tools():
  """Gets tools from the Customer MCP Server."""
  print("Attempting to connect to Customer MCP server...")
  
  mcp_base_url = f"https://api-{CLOUD_PROJECT}.apiservices.dev/crm-mcp-proxy/sse"
  if not mcp_base_url:
    raise ValueError("Variable MCP_BASE_URL is not set.")

  client_id = "CRM_TOOLS_API_KEY"  # @param {type:"string"}

  if not client_id:
    raise ValueError("Environment variable CRM_TOOLS_API_KEY is not set.")

  tools_packed = MCPToolset(
      connection_params=SseServerParams(
          url=mcp_base_url,
          headers={"x-api-key": client_id},
      )
  )
  print("MCP Customer Toolset created successfully.")
  return tools_packed

### Initialize Agent

In [60]:
def get_customer_agent():
    """Creates a Customer Agent"""
    tools = get_customer_tools()
    customer_agent = Agent(
        model="gemini-2.5-pro-preview-05-06",
        name="customer_agent",
        description="Central authority for all customer account data and related engagement triggers",
        instruction="""
        - You are responsible for managing customer account data. Your primary function is to use the available tool operations to accurately and securely handle customer information.
          - **Manage user profiles:**
            - Retrieve comprehensive profile details for a specific customer using their customer ID. This can be used for purposes like pre-filling order forms.
          - **Manage customer addresses:**
            - Retrieve a list of all saved addresses for a specific customer.
            - Retrieve the details of a specific address for a customer.
        """,
        tools=[tools],
    )
    return customer_agent

### Build Agent Runtime

In [61]:
APP_NAME="crm_app"
USER_ID="user_fs"
SESSION_ID="user_session"

async def async_main():
  session_service = InMemorySessionService()
  artifacts_service = InMemoryArtifactService()

  session = await session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
  )
  query = "get details about customer 1234"
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent = get_customer_agent()

  runner = Runner(
      app_name='crm_app',
      agent=root_agent,
      artifact_service=artifacts_service,
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=SESSION_ID, user_id=USER_ID, new_message=content
  )

  async for event in events_async:
    print(f"Event received: {event}")

  print("Cleanup complete.")

### Execute Agent

In [62]:
await async_main()

User Query: 'get details about customer 1234'
Attempting to connect to Customer MCP server...
MCP Customer Toolset created successfully.
Running agent...


  + Exception Group Traceback (most recent call last):
  |   File "/Users/gonzalezruben/Documents/apigee-adk-workshop/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3668, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_77431/2229161767.py", line 1, in <module>
  |     await async_main()
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_77431/4242907094.py", line 30, in async_main
  |     async for event in events_async:
  |       print(f"Event received: {event}")
  |   File "/Users/gonzalezruben/Documents/apigee-adk-workshop/.venv/lib/python3.13/site-packages/google/adk/runners.py", line 196, in run_async
  |     async for event in invocation_context.agent.run_async(invocation_context):
  |     ...<2 lines>...
  |       yield event
  |   File "/Users/gonzalezruben/Documents/apigee-adk-workshop/.venv/lib/python3.13/site-packages/google/adk/a